## We start with an example of verification Job document
```
$ dispatch_verify.py --test --env jenkins --lsstsw $(pwd) Cfht_output_r.json --write verify_job.json

```

In [54]:
import json
import requests
with open("data/verify_job.json", "r") as f:
     data = json.load(f)

A Job is a container for measurements, metadata, data blobs as well as for metrics and metric specifications. Metrics and metric specifications are pre-loaded to SQuaSH and only measurements, metadata and data blobs are sent to SQuaSH each time a new verification job is registered. For the purpose of testing the API we will exercise loading metrics and specifications here too.

In [55]:
data.keys()

dict_keys(['metrics', 'measurements', 'meta', 'blobs', 'specs'])

## Here we load a list of metrics to SQuaSH
We assume you are running the `squash-rest-api` service locally, or you could use the URL https://squash-api-demo.lsst.codes, if available.

You can check the API documentation at http://localhost:5000/apidocs 

In [56]:
data['metrics']

[{'description': 'The averaged E1, E2, and Ex residual PSF ellipticity correlations must have median absolute value less than this for separations < 1 arcmin.',
  'name': 'validate_drp.TE1',
  'reference': {'doc': 'LPM-17', 'page': 31, 'url': 'http://ls.st/lpm-17'},
  'tags': [],
  'unit': ''},
 {'description': 'The maximum fraction of astrometric distances which deviate by more than AD3 milliarcsec (see AM3).',
  'name': 'validate_drp.AF3_stretch',
  'reference': {'doc': 'LPM-17', 'page': 23, 'url': 'http://ls.st/lpm-17'},
  'tags': [],
  'unit': '%'},
 {'description': 'Total time elapsed in AstrometryTask.',
  'name': 'meas_astrom.AstrometryTime',
  'reference': {'doc': None,
   'page': None,
   'url': 'https://confluence.lsstcorp.org/pages/viewpage.action?spaceKey=~ebellm&title=Alert+Production+Metrics'},
  'tags': ['ap_verify', 'task', 'monitoring', 'workflow'],
  'unit': 's'},
 {'description': 'Total time elapsed in CharacterizeImageTask.',
  'name': 'pipe_tasks.CharacterizeImageT

In [57]:
metrics={'metrics': data['metrics']}
r = requests.post("http://localhost:5000/metrics", json=metrics)
r.json()


{'message': 'List of metrics successfully loaded.'}

## And the metric specifications

In [58]:
data['specs']

[{'metadata_query': {},
  'name': 'release.AD3.FY19',
  'tags': [],
  'threshold': {'operator': '<=', 'unit': 'marcsec', 'value': 30.0},
  'type': 'threshold'},
 {'metadata_query': {'filter_name': 'any', 'instrument': 'HSC'},
  'name': 'validate_drp.AM2.hsc_design',
  'tags': ['AM2'],
  'threshold': {'operator': '<=', 'unit': 'marcsec', 'value': 10.0},
  'type': 'threshold'},
 {'metadata_query': {'filter_name': 'any', 'instrument': 'HSC'},
  'name': 'validate_drp.AM3.hsc_minimum',
  'tags': ['AM3'],
  'threshold': {'operator': '<=', 'unit': 'marcsec', 'value': 30.0},
  'type': 'threshold'},
 {'metadata_query': {'dataset_repo_url': 'https://github.com/lsst/validation_data_cfht.git',
   'filter_name': 'i',
   'instrument': 'CFHT'},
  'name': 'release.PA1.FY20_cfht_i',
  'tags': [],
  'threshold': {'operator': '<=', 'unit': 'mmag', 'value': 8.0},
  'type': 'threshold'},
 {'metadata_query': {'dataset_repo_url': 'https://github.com/lsst/validation_data_hsc.git',
   'filter_name': 'HSC-G',
 

In [59]:
specs={'specs': data['specs']}
r = requests.post("http://localhost:5000/specs", json=specs)
r.json()


{'message': 'List of metric specificationss successfully created.'}

## Finally we load a verification job  with measurements, metadata and data blobs


Job metatada consists of (arbitrary) metadata, packages and environment metadata

In [60]:
data['meta'].keys()

dict_keys(['instrument', 'packages', 'dataset_repo_url', 'env', 'filter_name'])

In [61]:
data['meta']['env']

{'ci_dataset': 'unknown',
 'ci_id': 'unknown',
 'ci_label': 'unknown',
 'ci_name': 'unknown',
 'ci_url': 'https://example.com',
 'date': '2017-12-28T18:29:36.555029+00:00',
 'env_name': 'jenkins',
 'status': 0}

In [62]:
data['meta']['packages']

{'apr': {'eups_version': '1.5.2',
  'git_branch': None,
  'git_sha': '39b3212aa46217e4f485b02496381907da8b8d7a',
  'git_url': 'https://github.com/lsst/apr.git',
  'name': 'apr'},
 'apr_util': {'eups_version': '1.5.4',
  'git_branch': None,
  'git_sha': '45d74ccc249484be4fdba93fe2f66aafb5869335',
  'git_url': 'https://github.com/lsst/apr_util.git',
  'name': 'apr_util'},
 'astropy': {'eups_version': '2.0.1',
  'git_branch': None,
  'git_sha': '4d079c1f2ee2e98c7771a0c61b15ffd17e211257',
  'git_url': 'https://github.com/lsst/astropy.git',
  'name': 'astropy'},
 'base': {'eups_version': '14.0-8-g7f6dd6b',
  'git_branch': None,
  'git_sha': '7f6dd6b394bba8c4305053c87ee61f1a266b4dd6',
  'git_url': 'https://github.com/lsst/base.git',
  'name': 'base'},
 'boost': {'eups_version': '1.60.lsst1+2',
  'git_branch': None,
  'git_sha': '18cc8bacfbd8ac16a414ce6acb450e85ee174ca5',
  'git_url': 'https://github.com/lsst/boost.git',
  'name': 'boost'},
 'doxygen': {'eups_version': '1.8.13.lsst1',
  'git_

In [63]:
job = {'measurements': data['measurements'], 'meta': data['meta'], 'blobs': data['blobs']}
r = requests.post("http://localhost:5000/job", json=job)
r.json()

{'message': 'Job `1` successfully created'}

## Test job deletion

In [64]:
job_id = 1
r = requests.delete("http://localhost:5000/job/{}".format(job_id))
r.json()

{'message': 'Job successfully deleted.'}